### `Participation Rate` vs `Conditional Participation Rate`.

to help understand the different summing of rates

In [1]:
#import pathlib
import json
import nzshm_model as nm
import solvis

from solvis.filter import FilterRuptureIds, FilterParentFaultIds, FilterSubsectionIds


warning openquake module dependency not available, maybe you want to install
                with nzshm-model[openquake]


Running without `toshi` options


In [2]:
# OPTIONS
SINGLE_SOLUTION = True
SECTION_RATE = False # otherwise Parent rates

if SINGLE_SOLUTION:
    solution = solvis.InversionSolution.from_archive("NZSHM22_ScaledInversionSolution-QXV0b21hdGlvblRhc2s6MTEzMTM0.zip")
else:
    current_model = nm.get_model_version("NSHM_v1.0.4")
    slt = current_model.source_logic_tree
    csol = solvis.CompositeSolution.from_archive("NSHM_v1.0.4_CompositeSolution.zip", slt)
    solution = csol._solutions['CRU']

# # helper = FaultSystemSolutionHelper(solution)  
rate_column = "Annual Rate" if isinstance(solution, solvis.InversionSolution) else "rate_weighted_mean"


In [3]:
# get ruptures passing through a named parent fault
TARGET_FAULTS = ['Opotiki 3', 'Calypso 2']
# TARGET_FAULTS = ['Opotiki 3'] 

# ruptures = helper.ruptures_for_parent_fault_names(TARGET_FAULTS) # 
ruptures = FilterRuptureIds(solution)\
    .for_parent_fault_names(TARGET_FAULTS)


### calculate participation rate series

In [4]:
# for comparison
opo_section = 1426 # Opotiki 3, Subsection 2
cal_section = 217  # Calypso 2, Subsection 1
df0 = solution.rs_with_rupture_rates.copy()

print("summing the subsection rate")
print("===========================")

print("new sum() filtering by section ID: `Participation Rate` (all ruptures in the system)")
print(f'Opotiki rate: {df0[df0.section==opo_section][rate_column].sum()}')
print(f'Calypso rate: {df0[df0.section==cal_section][rate_column].sum()}')
print()

summing the subsection rate
new sum() filtering by section ID: `Participation Rate` (all ruptures in the system)
Opotiki rate: 0.0001605307770660147
Calypso rate: 0.0002327073598280549



In [5]:
opsr = solution.section_participation_rates([opo_section, cal_section])
opsr

,Rupture Index,participation_rate
section,,
217,401175184,0.000233
1426,451122703,0.000161


In [6]:
TARGET_FAULTS = ['Opotiki 2', 'Opotiki 3', 'Calypso 2']
ruptures = FilterRuptureIds(solution)\
    .for_parent_fault_names(TARGET_FAULTS)
    # .for_magnitude(min_mag=5.1, max_mag=9.4)

print(list(ruptures))

[113538, 113540, 113194, 265419, 312490, 357323, 367099, 275100, 113533, 322207]


In [7]:
fault_ids = FilterParentFaultIds(solution).for_parent_fault_names(TARGET_FAULTS)
opsr = solution.fault_participation_rates(fault_ids, ruptures)
opsr

,participation_rate
ParentID,
52,0.000233
329,0.000064
330,0.000161
